In [1]:
import pandas as pd
import numpy as np

import helper
import project_helper
import project_tests

# Market Data

In [2]:
df = pd.read_csv('./data/eod-quotemedia.csv')

percent_top_dollar = 0.2
high_volume_symbols = project_helper.large_dollar_volume_stocks(df, 'adj_close', 'adj_volume', percent_top_dollar)
df = df[df['ticker'].isin(high_volume_symbols)]

close = df.reset_index().pivot(index='date', columns='ticker', values='adj_close')
volume = df.reset_index().pivot(index='date', columns='ticker', values='adj_volume')
dividends = df.reset_index().pivot(index='date', columns='ticker', values='dividends')

In [3]:
project_helper.print_dataframe(close)

# Part 1: Smart Beta Portfolio

## Index Weights

In [4]:
def generate_dollar_volume_weights(close, volume):
    """
    Generate dollar volume weights.

    Parameters
    ----------
    close : DataFrame
        Close price for each ticker and date
    volume : str
        Volume for each ticker and date

    Returns
    -------
    dollar_volume_weights : DataFrame
        The dollar volume weights for each ticker and date
    """
    assert close.index.equals(volume.index)
    assert close.columns.equals(volume.columns)
    
    product = close * volume
    
    result = product.copy()
    
    for index, row in result.iterrows():
        result.at[index] = row/np.sum(row)

    return result

In [5]:
project_tests.test_generate_dollar_volume_weights(generate_dollar_volume_weights)

Tests Passed


In [6]:
index_weights = generate_dollar_volume_weights(close, volume)
project_helper.plot_weights(index_weights, 'Index Weights')

# Portfolio Weights

In [7]:
def calculate_dividend_weights(dividends):
    """
    Calculate dividend weights.

    Parameters
    ----------
    dividends : DataFrame
        Dividend for each stock and date

    Returns
    -------
    dividend_weights : DataFrame
        Weights for each stock and date
    """
    
    result = dividends.copy()
    
    dividend_sum = np.zeros(len(dividends.columns))
    
    for index, row in result.iterrows():
        
        dividend_sum += row
        
        result.at[index] = dividend_sum / np.sum(dividend_sum)

    return result

In [8]:
project_tests.test_calculate_dividend_weights(calculate_dividend_weights)

Tests Passed


In [9]:
etf_weights = calculate_dividend_weights(dividends)
project_helper.plot_weights(etf_weights, 'ETF Weights')

# Returns

In [10]:
def generate_returns(prices):
    """
    Generate returns for ticker and date.

    Parameters
    ----------
    prices : DataFrame
        Price for each ticker and date

    Returns
    -------
    returns : Dataframe
        The returns for each ticker and date
    """
    
    return (prices - prices.shift(1)) / prices.shift(1)

In [11]:
project_tests.test_generate_returns(generate_returns)

Tests Passed


In [12]:
returns = generate_returns(close)
project_helper.plot_returns(returns, 'Close Returns')

# Weighted Returns

In [13]:
def generate_weighted_returns(returns, weights):
    """
    Generate weighted returns.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date
    weights : DataFrame
        Weights for each ticker and date

    Returns
    -------
    weighted_returns : DataFrame
        Weighted returns for each ticker and date
    """
    assert returns.index.equals(weights.index)
    assert returns.columns.equals(weights.columns)

    return returns * weights

In [14]:
project_tests.test_generate_weighted_returns(generate_weighted_returns)

Tests Passed


In [15]:
index_weighted_returns = generate_weighted_returns(returns, index_weights)
etf_weighted_returns = generate_weighted_returns(returns, etf_weights)
project_helper.plot_returns(index_weighted_returns, 'Index Returns')
project_helper.plot_returns(etf_weighted_returns, 'ETF Returns')

# Cumulative Returns

In [16]:
def calculate_cumulative_returns(returns):
    """
    Calculate cumulative returns.

    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date

    Returns
    -------
    cumulative_returns : Pandas Series
        Cumulative returns for each date
    """
    
    results = (1 + returns.sum(axis=1, skipna=True)).cumprod()
    
    results[0] = np.nan
   
    return results

In [17]:
project_tests.test_calculate_cumulative_returns(calculate_cumulative_returns)

Tests Passed


In [18]:
index_weighted_cumulative_returns = calculate_cumulative_returns(index_weighted_returns)
etf_weighted_cumulative_returns = calculate_cumulative_returns(etf_weighted_returns)
project_helper.plot_benchmark_returns(index_weighted_cumulative_returns, etf_weighted_cumulative_returns, 'Smart Beta ETF vs Index')